In [1]:
import os
import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# 忽略警告
warnings.filterwarnings('ignore')

# Matplotlib 配置
plt.rcParams.update({
    "pgf.texsystem": "xelatex",
    'font.family': 'serif',
    'text.usetex': False,
    'pgf.rcfonts': False,
    'figure.dpi': 300,
})

# 自动重载
%load_ext autoreload
%autoreload 2

# 设置包路径，指向本仓库 src 目录
package_path = r'C:\Users\Mingchuan\Huanglab\spatial\PRISM\code\src'
if package_path not in sys.path:
    sys.path.append(package_path)

from gene_calling.projection import (
    projection_gene,
    plot_params_generator,
    export_density_tiff,
    generate_colormap
)

BASE_DIR = Path(r'G:\spatial_data\processed')
RUN_ID = '20250318_ZP_YCXin_PRISM_pro_exp_4_con_1_EtOH_Ab'
src_dir = BASE_DIR / RUN_ID
stc_dir = src_dir / 'stitched'
read_dir = src_dir / 'readout'

## plt format

In [2]:
mapped_genes = pd.read_csv(read_dir / 'mapped_genes.csv')
gene_list = mapped_genes['Gene'].unique()

out_path = read_dir / 'density_png'
out_path.mkdir(exist_ok=True)

In [3]:
from tifffile import TiffFile
with TiffFile(stc_dir/'cyc_1_cy3.tif') as tf:
    ref_shape = tf.pages[0].shape  # (rows, cols)

# 使用内置的色图生成函数
thermal_LUT = pd.read_csv(Path(package_path)/'gene_calling'/'thermal_LUT.csv', index_col=0)
thermal_LUT.loc[0] = [0, 0, 0]
cmap_thermal = generate_colormap(thermal_LUT.values)
# cmap_thermal

plot_params_update = {
    'canvas_shape': ref_shape,
    'downsample_factor': 100,
    'cmap': cmap_thermal, 
    # 'vmin': 2, 'vmax': 8,
    'export_full_png': True,
    'export_main_png': True,
    # 'scale_bar': {
    #   'pixel_size_um': 0.1625,    # 原始像素大小（可改）
    #   'length_um': 1000,           # 比例尺长度
    #   'height_px': 10,
    #   'color': 'white',
    #   'margin_px': 20,
    #   'fontsize': 12,
    #   'label': '',
    # }
    }

In [4]:
gene_order = 2
gene_name = gene_list[gene_order-1]
print(gene_name)
points = mapped_genes[mapped_genes['Gene']==gene_name][['X', 'Y']].values
x = points[:, 0]
y = points[:, 1]

projection_gene(x=x, y=y, gene_name=gene_name, outpath=out_path,
                plot_params_update=plot_params_update)

Slc17a7


In [5]:
for gene_order in tqdm(range(1, len(gene_list)+1)):
  gene_name = gene_list[gene_order-1]
  points = mapped_genes[mapped_genes['Gene']==gene_name][['X', 'Y']].values
  x = points[:, 0]
  y = points[:, 1]
  projection_gene(x, y, gene_name=gene_name, outpath=out_path,
                  plot_params_update=plot_params_update)

100%|██████████| 45/45 [01:07<00:00,  1.50s/it]


# tiff images

In [6]:
from glob import glob

density_dir = read_dir / 'density_tiff'
density_dir.mkdir(exist_ok=True)

# 使用库函数按参考TIFF尺寸导出下采样密度图
mapped_genes = pd.read_csv(read_dir / 'mapped_genes.csv')
gene_list = mapped_genes['Gene'].unique()

ref_tif = stc_dir / 'cyc_1_cy3.tif'
for gene in tqdm(gene_list):
    if glob(str(density_dir / f'{gene}.tif')):
        continue
    export_density_tiff(
        mapped_genes[mapped_genes['Gene'] == gene],
        density_dir / f'{gene}.tif',
        ref_shape=ref_shape,   # 直接用 shape
        y_col='Y', x_col='X', fac=100
    )

100%|██████████| 45/45 [03:06<00:00,  4.15s/it]
